## AI Inference from a video file into a bunch of images
This notebook is an example how to use DeGirum PySDK to do AI inference of a video stream from a video file. The annotated frames are saved into new directory with `_annotated` suffix.

This script works with the following inference options:

1. [DeGirum Cloud Platform](https://cs.degirum.com),
1. DeGirum-hosted AI server node shared via Peer-to-Peer VPN,
1. AI server node hosted by you in your local network,
1. AI server running on your local machine,
1. DeGirum ORCA accelerator directly installed on your local machine.

To try different options, you just need to change the `inference_option` in the code below.

### Specify where do you want to run your inferences and video file name here

In [ ]:
inference_option = 1  # <<< change it according to your needs selecting from the list in the header comment
input_filename = "./images/Traffic.mp4" # video file to process

### The rest of the cells below should run without any modifications

In [ ]:
import degirum as dg # import DeGirum PySDK
import mytools, cv2
from pathlib import Path

In [ ]:
# connect to model zoo according to selected inference option
zoo = mytools.connect_model_zoo(inference_option)

In [ ]:
# load object detection model
# (change model name to "...n2x_cpu_1" to run it on CPU)
model = zoo.load_model("mobilenet_v2_ssd_coco--300x300_quant_n2x_orca_1")

# set model parameters
model.image_backend = 'opencv' # select OpenCV backend: needed to have overlay image in OpenCV format
model.input_numpy_colorspace = 'BGR'
model.overlay_show_probabilities = True # show probabilities on overlay image
model.overlay_line_width = 1

In [ ]:
orig_path = Path(input_filename)
frame_path = orig_path.parent / (orig_path.name + "_original") # construct original images directory path

# decode video stream into a set of jpeg images
nframes = mytools.video2jpegs(input_filename, frame_path)
print(f"{nframes} images are saved into '{frame_path}'")

In [ ]:
ann_path = orig_path.parent / (orig_path.name + "_annotated") # construct annotated images directory path

if not ann_path.exists(): # create directory for annotated images
    ann_path.mkdir()
    
# AI prediction loop
fi = 0 # frame index
print("Processing frames:")
progress = mytools.Progress(nframes)
for res in model.predict_dir(str(frame_path)):
    fname = str(ann_path / f"frame_{fi:05d}.jpg")
    cv2.imwrite(fname, res.image_overlay)
    fi += 1
    progress.step()
print(f"{nframes} annotated images are saved into '{ann_path}'")